<a href="https://colab.research.google.com/github/beyg1/Q3/blob/main/LangChain_ToolCall_Calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install -q -U langchain-google-genai # google's generative ai installation with langchain
from google.colab import userdata #importing gemini key
from langchain_google_genai import ChatGoogleGenerativeAI

                                     #connecting LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",api_key=userdata.get('GOOGLE_API_KEY'))
llm.invoke("Yello")

                                    #Make Tools/Functions using @tool
from langchain_core.tools import tool

@tool
def add(i:int,j:int):
  """Adds two numbers"""
  return i+j
@tool
def minus(i:int,j:int):
  """Subtracts two numbers"""
  return i-j
@tool
def multiply(i:int,j:int):
  """Returns Product of two numbers"""
  return i*j
@tool
def divide(i:int,j:int):
  """Returns Division of two numbers"""
  return i/j

tools = [add,minus,multiply,divide]

llm_tools = llm.bind_tools(tools) ## Bind tools to the LLM

In [44]:
from IPython.display import display
#The display function is used to nicely present outputs within a Jupyter Notebook environment.

# create message(Prompts & Responses) history
from langchain_core.messages import HumanMessage, AIMessage
query = "Multiplication of 4 & 5"
messages = [HumanMessage(query)]
display(messages)
#4 kinds of messages: Human message, System message, AI message, Tool message


[HumanMessage(content='Multiplication of 4 & 5', additional_kwargs={}, response_metadata={})]

In [45]:
ai_msg_tools = llm_tools.invoke(messages)
messages.append(ai_msg_tools)
display(messages)
# Remember messages is our list of messages in the conversation? This line adds
# the LLM's response (ai_msg_tools) to that list. So now messages has both our
# original query and the LLM's answer. This is useful for keeping track of the conversation history.

[HumanMessage(content='Multiplication of 4 & 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"i": 4.0, "j": 5.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-949a37d7-94b1-4dba-aed8-147705c21624-0', tool_calls=[{'name': 'multiply', 'args': {'i': 4.0, 'j': 5.0}, 'id': 'c8f5cba2-332e-4ab5-85c3-2cb9096c36b5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 190, 'output_tokens': 3, 'total_tokens': 193, 'input_token_details': {'cache_read': 0}})]

In [46]:
ai_msg_tools.tool_calls #Confirms the tool called by llm

[{'name': 'multiply',
  'args': {'i': 4.0, 'j': 5.0},
  'id': 'c8f5cba2-332e-4ab5-85c3-2cb9096c36b5',
  'type': 'tool_call'}]

In [47]:
#Finally to get AI message, in this case tool message, which will be passed to llm:
for tool_call in ai_msg_tools.tool_calls:
    selected_tool = {
        "add": add,
        "minus": minus,
        "multiply": multiply,
        "division": divide,
      }[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    display(tool_msg)
    messages.append(tool_msg) #tool message added in chat history

ToolMessage(content='20', name='multiply', tool_call_id='c8f5cba2-332e-4ab5-85c3-2cb9096c36b5')

In [48]:
display(messages)
#prompt (human message) and tool message is now saved in messages

[HumanMessage(content='Multiplication of 4 & 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"i": 4.0, "j": 5.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-949a37d7-94b1-4dba-aed8-147705c21624-0', tool_calls=[{'name': 'multiply', 'args': {'i': 4.0, 'j': 5.0}, 'id': 'c8f5cba2-332e-4ab5-85c3-2cb9096c36b5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 190, 'output_tokens': 3, 'total_tokens': 193, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='20', name='multiply', tool_call_id='c8f5cba2-332e-4ab5-85c3-2cb9096c36b5')]

In [49]:
# Invoke the llm with messages
response = llm_tools.invoke(messages)
messages.append(response)

In [50]:
#for a nice display format
from IPython.display import Markdown

Markdown(response.content)

The result of the multiplication is 20.